## Подключаем основные библиотеки

In [1]:
import pandas as pd
import numpy as np

Загружаем датасет из предыдущего релиза

In [2]:
msk = pd.read_csv('../realises2/data.csv')

In [3]:
msk.head(1)

,id,number_of_rooms,metro,adress,apartment_area,house,parking,price_per_month,repair,balcony,windows,bathroom,child_or_animals,ceiling_height,elevator,garbage_chute
0,271271157,4,Рядом,"Москва, улица Новый Арбат, 27",200,"5/16, Монолитный",подземная,500000,Дизайнерский,"Балкон (0), Лоджия (0)",Во двор,Совмещенный (1),"Можно с детьми, Можно с животными",3.0,"Пасс (4), Груз (1)",Да


Удаляем адрес, так как нет возможности перевести его в числовой вид. Была идея, но не было времени...

In [4]:
msk.drop(['adress'], axis=1, inplace=True)

Разделяем колонку ДОМ на 2. Одна - тип дома, сделал с помощью регекса, мне очень нравится)

In [5]:
msk['house_type'] = msk['house'].str.extract(r',\s*([\w\s]+)$')

Вторая колонка - этажи. Тут, думаю, всё просто и понятно


In [6]:
def floors(val):
    part = val.split(',')
    if len(part) > 0:
        floor = part[0].strip()
        if '/' in floor:
            return floor
    return 0

msk['floor'] = msk['house'].apply(floors)

Удаляем изначальную колонку, зачем она нам??

In [7]:
msk.drop('house', axis=1, inplace=True)

In [8]:
msk['house_type'].value_counts().sum()

np.int64(15894)

In [9]:
msk['floor'].value_counts()

floor
3/5      524
4/5      511
2/5      481
5/5      433
5/9      393
        ... 
10/29      1
19/29      1
7/29       1
41/60      1
25/42      1
Name: count, Length: 884, dtype: int64

Переводим новые колонки в числовой вид. Идея какая, самые приятные для проживания этажи - те, что в середине. Отсюда и вывод, что для них нам нужно сделать что-то типа "нормального распределения". Ниже я написал эту функцию))) 

In [10]:
#преобразование floor к нормальному распределению. Чем центрее - тем лучше
def floors_to_norma(val):
    try:
        curr, vsego = map(int, val.split('/'))
        mid = vsego // 2

        if abs(curr - mid) <= 1:
            return 1
        elif curr == 1 or curr == vsego:
            return 0.2
        else:
            diff = abs(curr - mid)
            return max(0.2, 1 - (diff/mid))
    except ValueError:
        return 0
    
msk['floor'] = msk['floor'].apply(floors_to_norma)

Тут просто мапом переводим все полученные значения. Потом выяснится, что есть что-то ещё, их мы заполним просто средним значением

In [11]:
house_type = {
    'Панельный': 10,
    'Кирпичный': 12,
    'Монолитный': 9,
    'Блочный': 8,
    'Сталинский': 6,
    'старый фонд': 2, 
    'Деревянный': 1,
    'Щитовой': 1
}

msk['house_type'] = msk['house_type'].map(house_type)

In [12]:
msk['house_type'] = msk['house_type'].fillna(4)

In [13]:
msk['house_type'].isna().sum()

np.int64(0)

Смотрим, что у нас тут по лифтам. Задача - Разделить на 2 столбца (Грузовой и пассажирский) и в каждый занести значения из скобок. Если одного из лифтов нет - заполнить отсутствующее значение 0

In [14]:
msk['elevator'].value_counts()

elevator
Пасс (1)               9057
Пасс (2)               3997
Пасс (1), Груз (1)     3431
Пасс (2), Груз (1)     1091
Пасс (2), Груз (2)      606
Пасс (3)                545
Пасс (4)                307
Пасс (3), Груз (1)      170
Пасс (1), Груз (2)      168
Пасс (4), Груз (2)       60
Груз (1)                 57
Пасс (4), Груз (1)       55
Пасс (3), Груз (3)       42
Пасс (3), Груз (2)       36
Груз (4)                 25
Пасс (4), Груз (4)       23
Груз (2)                 18
Груз (3)                 15
Пасс (4), Груз (3)        7
Пасс (6)                  6
Пасс (1), Груз (3)        5
Пасс (60)                 2
Пасс (2), Груз (3)        2
Пасс (50)                 2
Пасс (5), Груз (1)        1
Пасс (1), Груз (12)       1
Пасс (8), Груз (8)        1
Пасс (7)                  1
Пасс (1), Груз (4)        1
Пасс (5), Груз (3)        1
Пасс (2), Груз (4)        1
Груз (6)                  1
Груз (8)                  1
Пасс (5)                  1
Name: count, dtype: int64

2 функции. Одна ищет и заполняет 

In [15]:
#Функция для пассажирских
def passanger(val):
    if val:
        if 'Пасс' in val:
            try:
                return int(val.split('Пасс (')[1].split(')')[0])
            except:
                return 0
    return 0

def gruz(val):
    if val:
        if 'Груз' in val:
            try:
                return int(val.split('Груз (')[1].split(')')[0])
            except:
                return 0
    return 0

msk['elevator_pass'] = msk['elevator'].apply(passanger)
msk['elevator_gruz'] = msk['elevator'].apply(gruz)

Удаляем Колонку лифтов за ненадобностью

In [16]:
msk.drop('elevator', axis=1, inplace=True)

Меняем значения в "Метро" и "мусоропроводе на числовые"

In [17]:
msk['metro'] = msk['metro'].str.contains('Рядом').astype(int)
msk['garbage_chute'] = msk['garbage_chute'].str.contains('Да').astype(int)

Для парковочных мест придумываем численные значения


In [18]:
mapping = {
    'наземная': 1,
    'подземная': 2,
    'открытая': 3,
    'многоуровневая': 4,
    'на крыше': 5
}

msk['parking'] = msk['parking'].map(mapping)

То же самое для ремонта. Без ремонта - плохо, а с ним уже лучше, но зависит от его типа

In [19]:
repair_apart = {
    'Без ремонта': 1,
    'Косметический': 2,
    'Евроремонт': 3,
    'Дизайнерский': 4
}
msk['repair'] = msk['repair'].map(repair_apart)

Также приводим к численному значению вид из окна. Мы решили, если во двор - то не очень хорошо, если на улицу, то лучше - допустим, там парк/лес/река и тд. Ну а если у нас сквозная квартира, то вообще отлично!

In [20]:
map_dwor = {
    'Во двор': 1,
    'На улицу и двор': 3,
    'На улицу': 2
}
msk['windows'] = msk['windows'].map(map_dwor)

Переводим балконы в числа! 


In [21]:
def balkon(val):
    if val:
        if 'Балкон' in val:
            try:
                return int(val.split('Балкон (')[1].split(')')[0])
            except:
                return 0
    return 0

def lodg(val):
    if val:
        if 'Лоджия' in val:
            try:
                return int(val.split('Лоджия (')[1].split(')')[0])
            except:
                return 0
    return 0

msk['balkon'] = msk['balcony'].apply(balkon)
msk['lodgy'] = msk['balcony'].apply(lodg)
msk.drop('balcony', axis=1, inplace=True)

И к нашим туалетам, конечно же!

In [22]:
msk['bathroom'].value_counts()

bathroom
Совмещенный (1)                    10541
Раздельный (1)                      6139
Совмещенный (2)                     1293
Совмещенный (1), Раздельный (1)      730
Раздельный (2)                       401
Совмещенный (3)                      223
Совмещенный (2), Раздельный (1)      179
Совмещенный (4)                       74
Раздельный (3)                        44
Совмещенный (1), Раздельный (2)       28
Совмещенный (3), Раздельный (1)       26
Совмещенный (2), Раздельный (2)       22
Раздельный (4)                        15
Совмещенный (3), Раздельный (3)        6
Совмещенный (4), Раздельный (1)        6
Совмещенный (4), Раздельный (2)        4
Совмещенный (1), Раздельный (3)        2
Совмещенный (2), Раздельный (3)        2
Совмещенный (2), Раздельный (4)        1
Совмещенный (3), Раздельный (2)        1
Name: count, dtype: int64

Делаем абсолютно то же самое, что и для балконов и лифтами

In [23]:
def toilet(val):
    if val:
        if 'Совмещенный' in val:
            try:
                return int(val.split('Совмещенный (')[1].split(')')[0])
            except:
                return 0
    return 0

def vanna(val):
    if val:
        if 'Раздельный' in val:
            try:
                return int(val.split('Раздельный (')[1].split(')')[0])
            except:
                return 0
    return 0

msk['sovm'] = msk['bathroom'].apply(toilet)
msk['razd'] = msk['bathroom'].apply(vanna)
msk.drop('bathroom', axis=1, inplace=True)

Изменяем значения для возможностей съема с кошкой/собакой

In [24]:
msk['child_or_animals'].value_counts()

map_child = {
    'Можно с детьми': 1,
    'Можно с детьми, Можно с животными': 3,
    'Не указано': 0,
    'Можно с животными': 2
}
msk['child_or_animals'] = msk['child_or_animals'].map(map_child)

Делаем таргет

In [25]:
msk['target'] = msk['price_per_month'].copy()
msk.drop(columns=['price_per_month'], axis=1, inplace=True)

Смотрим данные

In [ ]:
msk.head(10)

,id,number_of_rooms,metro,apartment_area,parking,repair,windows,child_or_animals,ceiling_height,garbage_chute,house_type,floor,elevator_pass,elevator_gruz,balkon,lodgy,sovm,razd,target
0,271271157,4,1,200,2,4,1,3,3.0,1,9.0,0.625,4,1,0,0,1,0,500000


Проверяем на наличие пропусков (ЧТОБЫ ТОЧНО)

In [31]:
msk.isna().sum()


id                  0
number_of_rooms     0
metro               0
apartment_area      0
parking             0
repair              0
windows             0
child_or_animals    0
ceiling_height      0
garbage_chute       0
house_type          0
floor               0
elevator_pass       0
elevator_gruz       0
balkon              0
lodgy               0
sovm                0
razd                0
target              0
dtype: int64

Удаляем дубликаты

In [28]:
msk = msk.drop_duplicates()

Выгружаем в файл

In [30]:
msk.to_csv('pylions.csv', index=False, encoding='utf-8')